<a href="https://colab.research.google.com/github/ankit-logiciel/reporing-metadata-generation/blob/main/JP_Metadata_file_generation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import json
from google.colab import files
import io


In [21]:
tables = {
    "v_customers": ("id","company_id","first_name","last_name","company_name","email","additional_emails","rep_id","referral_source","referral_existing_customer","referral_id","referral_type","referral_note","canvasser_notes","canvasser","call_center_rep_note","call_center_rep","customer_type","management_company","property_name","address_id","address","address_line_2","city","zip","state","country","lat","long","billing_address_id","billing_address","billing_address_line_2","billing_city","billing_zip","billing_state", "billing_country","billing_lat","billing_long","created_by_id","created_by","last_modified_by_id","modified_by","created_at","updated_at"),
    "v_jobs": ("id","company_id","customer_id","is_multi_job","name","job_id","job_number","lead_number","category_id","category","division_id","division","amount","tax_rate","address","address_line_2","city","zip","state","country","lat","long","current_stage","moved_to_current_state_at","job_lost_date","lost_job_marked_by","ageing","priority","purchase_order_number","awarded_prime_contractor_id","bid_customer_awarded_date","completion_date","archived_date_time","created_by_id","created_by","last_modified_by_id","modified_by","created_at","job_record_since","updated_at"),
    "v_projects": ("id","company_id","customer_id","name","job_id","number","parent_id","description","amount","tax_rate","job_lost_date","lost_job_marked_by","ageing","priority","purchase_order_number","other_trade_type_description","created_by_id","created_by","last_modified_by_id","modified_by","created_at","project_record_since","updated_at"),
    "v_proposals": ("id","company_id","job_id","title","is_file","note","status","shared_by","prime_contractor_id","worksheet_id","page_type","share_on_hop","share_on_hop_at","created_by_id","created_by","created_at","updated_at"),
    "v_workorders": ("id","company_id","title","template","job_id","type","worksheet_id","serial_number","is_file","total_amount","supplier_id","supplier_name","measurement_id","purchase_order_number","created_by","created_at","updated_at","deleted_at","deleted_by"),
    "v_job_schedules": ("id","company_id","job_id","title","description","repeat","series_id","occurence","interval","full_day","completed_at","start_datetime","end_datetime"),
    "v_job_sub_contractors": ("id","company_id","job_id","sub_contractor_id","schedule_id"),
    "v_appointments":("id","company_id","job_id","user_id","customer_id","title","description","location","lat","long","full_day","location_type","additional_recipient","repeat","occurence","until_date","interval","start_datetime","end_datetime","result","user_id","attendee","created_at","updated_at"),
    "v_change_orders": ("id","company_id","job_id","name","total_amount","tax_rate","taxable_amount","order","invoice_id","taxable","unit_number","division_id","divisionname","tier1","tier2","tier3","description","amount","quantity","product_name","product_id","trade_name","trade_id","work_type_id","work_type_nam","is_chargeable","supplier_name","supplier_id","branch_code","is_taxable","created_by_id","created_by","last_modified_by_id", "modified_by", "created_at","updated_at"),
    "v_job_invoices": ("id","company_id","customer_id","job_id","invoice_number","unit_number","title","name","description","due_date","billed_on","taxable","type","division_id","division_name","product_name","product_id","amount","description","quantity","work_type_name","work_type_id","trade","trade_id","is_chargeable","is_taxable","status","created_at","updated_at"),
    "v_job_payments": ("id","company_id","serial_number","job_id","payment","method","echeque_number","customer_id","status","date","ref_payment_id","ref_payment_to","created_by_id","created_by","last_modified_by_id","modified_by","created_at","updated_at"),
    "v_job_refunds": ("id","company_id","customer_id","job_id","financial_account_id","financial_account_name","payment_method","refund_number","refund_date","address","tax_amount","id","company_id","job_refund_id","financial_product_id","financial_product_name","quantity","rate","is_taxable","description","work_type_id","work_type_name","trade_id","trade_name","created_by_id","created_by","last_modified_by_id","modified_by","created_at","updated_at"),
    "v_vendor_bills": ("id","company_id","job_id","customer_id","vendor_id","vendor_name","bill_date","due_date","bill_number","note","total_amount","address","tax_amount","id","company_id","vendor_bill_id","financial_account_name","financial_account_id","rate","quantity","description","created_by_id","created_by","last_modified_by_id","modified_by","created_at","updated_at"),
    "v_credits": ("id","company_id","customer_id","job_id","amount","unapplied_amount","method","status","date","created_at","updated_at","ref_credit_id","ref_credit_to","reason"),
    "v_commissions": ("id","company_id","job_id","user_id","amount","due_amount","created_at","updated_at","commission_date","paid_on","paid_by","commission_percentage","status"),
    "v_commission_payments": ("id","company_id","job_id","commission_id","paid_by","amount","paid_on","created_at","updated_at"),
    "v_job_work_types": ("id", "company_id", "work_type_name", "work_type_id", "job_id" ), 
    "v_job_rep_estimator": ("id", "company_id", "job_id", "rep_id"),
    "v_job_company_crew": ("id", "company_id", "job_id", "rep_id"),
    "v_job_sub_contractor": ("id", "company_id", "job_id", "sub_contractor_id", "schedule_id", "schedule_recurring_id"),
    "v_job_follow_up": ("id", "company_id", "job_id", "rep_id", "customer_id", "mark"),
    "v_job_material_delivery_dates": ("id", "company_id", "job_id", "date"),
    "v_users": ("id","company_id","first_name","last_name","email","role_name","company_name","color","commission_percentage","all_division_assigned","user_profile_id","phone","address","address_line_1","city","state","country","zip","cell_number","home_number","fax_number","office_number","active","created_at","updated_at"),
    "v_user_divisions": ("id", "company_id", "division_id", "user_id", "division"),
    "v_timelogs": ("id","company_id","user_id","job_id","duration","start_date_time","end_date_time","clock_out_note","check_in_location","check_out_location","check_in_image","check_out_image","check_in_platform","check_out_platform","lat","long","check_out_lat","check_out_long","created_at","updated_at")
}

In [61]:

filename = "jp_reporting_schema(views).sql"

#  Upload the view etl file
# uploaded = files.upload()

# for fn in uploaded.keys():
#   filename = fn
#   print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  
try:
  f = open(filename, encoding = 'utf-8')
  content = f.read()
  content = content.split("CREATE VIEW ")
  del content[0]
  tables = {}
  for view in content:
  
    view_name = view.split("AS")[0]
    view_name = view_name.replace("\n", "")
 
    query = view.split("AS")[1]
    query = query.replace("SELECT", "").split("FROM")[0]
    fields = query.lower().split(" as ")
    field_names = []

    del fields[0]
    for field in fields:

      cleaned_name = field.split(",")[0]
      cleaned_name = cleaned_name.splitlines()[0]
      field_names.append(cleaned_name) 
    tables[view_name] = tuple(field_names)
finally:
  f.close()


In [62]:


def guess_field_config(field, config):
  manual_overrides = {
      'tax_rate': "CURRENCY",
      'archived_date_time': 'DATETIME',
      'is_file': "BOOLEAN"
  }

  #default settings to string
  datatype = "string"
  filter_type = "TEXT"
  field_type = "ATTRIBUTE"
  field_icon_type = "STRING"

  if field == "id" or "_id" in field or "_by" in field or (field in manual_overrides and manual_overrides[field] == "NUMBER"): 
    # apply number settings
    datatype = "integer"
    filter_type = "CONDITIONAL"
    field_type = "ATTRIBUTE"
    field_icon_type = "NUMBER"

  
  if field == "amount" or "_amount" in field or (field in manual_overrides and manual_overrides[field] == "CURRENCY"): 
    # apply currency settings
    datatype = "decimal"
    filter_type = "CONDITIONAL"
    field_type = "MEASURE"
    field_icon_type = "CURRENCY"

  if "_at" in field or "_since" in field or (field in manual_overrides and manual_overrides[field] == "DATETIME"): # apply datetime settings
    datatype = "datetime"
    filter_type = "DATE"
    field_type = "ATTRIBUTE"
    field_icon_type = "DATE_TIME"

  if field == "date" or field=="_date": # apply date settings
    datatype = "date"
    filter_type = "DATE"
    field_type = "ATTRIBUTE"
    field_icon_type = "DATE"

  if "is_" in field or (field in manual_overrides and manual_overrides[field] == "BOOLEAN"):
    datatype = "boolean"
    filter_type = "BOOLEAN"
    field_type = "ATTRIBUTE"
    field_icon_type = "BOOLEAN"

  config['datatype'] = datatype
  config['field_type'] = field_type
  config['field_icon_type'] = field_icon_type
  config['filter_type'] = filter_type

  return config


In [63]:
metadata = {
    "tables": []
}

for index, table in  enumerate(tables, start=1):
  table_meta = {}
  table_meta['name'] = table
  table_meta['alias'] = table
  table_meta['display_name'] = table.replace("v_", "").replace("_", " ").capitalize()
  table_meta['display_order'] = index
  table_meta['icon'] = "human"
  table_meta['fields'] = []
  for f_index, field in enumerate(tables[table], start=1):
    table_meta['fields'].append(guess_field_config(field, {
        "name": field,
        "display_name": field.replace("_", " ").capitalize(),
        "display_order": f_index,
        "help_text": "dummy",
    }))
  metadata['tables'].append(table_meta)

# https://jsoneditoronline.org/#left=local.vepoda&right=local.guqoye  
# format the json using this tool
json.dumps(metadata)


'{"tables": [{"name": "v_customers", "alias": "v_customers", "display_name": "Customers", "display_order": 1, "icon": "human", "fields": [{"name": "id", "display_name": "Id", "display_order": 1, "help_text": "dummy", "datatype": "integer", "field_type": "ATTRIBUTE", "field_icon_type": "NUMBER", "filter_type": "CONDITIONAL"}, {"name": "company_id", "display_name": "Company id", "display_order": 2, "help_text": "dummy", "datatype": "integer", "field_type": "ATTRIBUTE", "field_icon_type": "NUMBER", "filter_type": "CONDITIONAL"}, {"name": "first_name", "display_name": "First name", "display_order": 3, "help_text": "dummy", "datatype": "string", "field_type": "ATTRIBUTE", "field_icon_type": "STRING", "filter_type": "TEXT"}, {"name": "last_name", "display_name": "Last name", "display_order": 4, "help_text": "dummy", "datatype": "string", "field_type": "ATTRIBUTE", "field_icon_type": "STRING", "filter_type": "TEXT"}, {"name": "company_name", "display_name": "Company name", "display_order": 5,